In [1]:
import random

class Sentiment:
    POSITIVE="POSITIVE"
    NEUTRAL="NEUTRAL"
    NEGATIVE="NEGATIVE"


class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
        
    def get_sentiment(self):
        if self.score>=4:
            return Sentiment.POSITIVE
        elif self.score==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.NEGATIVE
        
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative=list(filter(lambda x:x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda x:x.sentiment==Sentiment.POSITIVE,self.reviews))
        positive_shrunk=positive[:len(negative)]
        self.reviews=negative+positive_shrunk
        random.shuffle(self.reviews)
        

In [2]:
import json

file_name="/Users/Admin/Desktop/Automotive_5.json"

reviews=[]
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Review(review['reviewText'],review["overall"]))

reviews[5].sentiment      
        

'POSITIVE'

In [3]:
#preprocessing

from sklearn.model_selection import train_test_split
training,test=train_test_split(reviews,test_size=0.3,random_state=42)

train_cont=ReviewContainer(training)
train_cont.evenly_distribute()
test_cont=ReviewContainer(test)
test_cont.evenly_distribute()

x_train=train_cont.get_text()
y_train=train_cont.get_sentiment()

x_test=test_cont.get_text()
y_test=test_cont.get_sentiment()

print(y_train.count(Sentiment.POSITIVE))
print(y_train.count(Sentiment.NEGATIVE))

800
800


In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer=TfidfVectorizer()
x_train_vector=vectorizer.fit_transform(x_train)
x_test_vector=vectorizer.transform(x_test)

x_train[0]


'This is a very simple product that works great. My RV fresh water intake in smaller then a metal hose end and I had to "kink" the hose to stop the flow of water (shut off valve was impractical) so filling the tank was a huge pain. I ended up getting soaked from "blow back" and had to stand and hold the hose while the 50 gal tank filled. The Valterra Quick Fill is a very simple product that\'s simply inserted into the fresh water filler neck. Turn on the valve and walk away. It stays inserted by itself so no more showers for me and I can go check the tires while the tank fills.  (Wish I\'d thought of it years ago)'

In [5]:
#linear svm


from sklearn import svm

clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(x_train_vector,y_train)

clf_svm.predict(x_test_vector[0])


array(['POSITIVE'], dtype='<U8')

In [6]:
print(clf_svm.score(x_test_vector,y_test))

0.7859195402298851


In [7]:
test_set=['this is very bad','good']
new_test=vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['NEGATIVE', 'POSITIVE'], dtype='<U8')

In [8]:
import pickle

In [9]:
pickle.dump(clf_svm,open('sa.pkl','wb'))

In [10]:
pickle.dump(vectorizer,open('vect.pkl','wb'))

In [11]:
r=pickle.load(open('vect.pkl','rb'))

In [13]:
clf_svm.predict(r.transform(['it is bad']))

array(['NEGATIVE'], dtype='<U8')